In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Загрузим данные

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv', parse_dates=[0])
mask = df.measurable_impressions > 0
df['CPM'] = df[mask].total_revenue / df[mask].measurable_impressions * 100000 #быстрее
df.CPM.fillna(0, inplace=True) # NaN там, где должно было быть деление на 0
df.drop(columns=[
    'total_revenue', #т.к. используем
    'order_id' , # как в оригинальной работе
    'line_item_type_id', # как в оригинальной работе
    'revenue_share_percent', # содержит одно значение
    'integration_type_id', # содержит одно значение
], inplace=True)
# df = df[(df.CPM >= 0) & (df.CPM < df.CPM.quantile(0.95))] #убираем выбросы
df.head(3)

In [ ]:
pd.concat((df.dtypes, df.isna().sum(0), df.describe().T), axis=1).rename(columns={0: 'type',1: 'NaNs'})

In [ ]:
id_cols = [x for x in df.columns if x.endswith('id')]
int_cols = [x for x in df.columns if not x.endswith('id') and 'date' != x and 'CPM' != x]
# df[id_cols] = df[id_cols].astype(str)
df[int_cols] = df[int_cols].astype(np.float)
id_cols, int_cols

# Фильтруем данные и делим на выборки

In [ ]:
df_train = df[df.date < pd.to_datetime("2019-06-22")].drop(columns='date')
df_test = df[df.date >= pd.to_datetime("2019-06-22")].drop(columns='date')


df_train = df_train[(df_train.CPM >= 0) & (df_train.CPM < df_train.CPM.quantile(0.95))]
df_test = df_test[(df_test.CPM >= 0) & (df_test.CPM < df.CPM.quantile(0.95))]
print(df_train.shape, df_test.shape)

Распределение targeta

In [ ]:
_, [ax1, ax2] = plt.subplots(1, 2, sharey=True, figsize=(10,6))
df_train.CPM.hist(ax=ax1)
df_test.CPM.hist(ax=ax2);

In [ ]:
df_train.describe().T

In [ ]:
df_test.describe().T

# Моделируем

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, FunctionTransformer #LabelEncoder
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error as mse
from xgboost import XGBRegressor
# from catboost import CatBoostRegressor

Сначала делал LabelEncoding, но в тестовой выборке не все пользователи присутствуют. По идее, нужно добавлять метку "неизвестный id". Не стал заморачиваться.

**TO DO**: добавить обработку неизвестных id

In [ ]:
# class MyLEncoder(LabelEncoder):

#     def transform(self, X, y=None, **fit_params):
#         enc_data = []
#         for i in range(self.size):
#             enc_data.append(self.enc[i].transform(X[X.columns[i]], **fit_params))
#         return np.asarray(enc_data).T

#     def fit_transform(self, X,y=None,  **fit_params):
#         self.fit(X, y,  **fit_params)
#         return self.transform(X, **fit_params)

#     def fit(self, X, y=None, **fit_params):
#         self.size = X.shape[1]
#         self.enc = []
#         for col in X.columns:
#             self.enc.append(LabelEncoder().fit(X[col], **fit_params))
#         return self

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df_train.drop(columns=['CPM']),
    df_train['CPM'],
    random_state=42
)
make_features = ColumnTransformer([
#     ('lbls', MyLEncoder(), id_cols),
    ('lbls', FunctionTransformer(lambda x: x.values), id_cols),
    ('ints', StandardScaler(), int_cols),
])
X_train = make_features.fit_transform(X_train)
X_val = make_features.transform(X_val)

In [ ]:
model = XGBRegressor(
    objective='reg:squarederror',
    n_jobs=-1, 
    random_state=42
)
model.fit(
    X = X_train, 
    y = y_train,
    eval_set = [(X_val, y_val)],
    verbose=0,
)
print('Validation mse: %.5f' % mse(y_val, model.predict(X_val)))

In [ ]:
model.feature_importances_

# Проверяем на тесте

In [ ]:
X_test, y_test = df_test.drop(columns=['CPM']),df_test['CPM']
X_test = make_features.transform(X_test)
test_mse = mse(y_test, model.predict(X_test))
print('Test mse: %.5f' % test_mse)
if test_mse > 4850: # Если вдруг что-то не так
    print('Khochy zachet!')